In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [3]:
def isvotes(tag):
    return tag.has_attr('class') and tag.has_attr('onclick') and not tag.has_attr('href') and not tag.has_attr('type')

In [35]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    try:
        soup = BeautifulSoup(page.text, 'html5lib')
        info = []
        if soup.find("span", class_= "post__title-text"):
            #id
            info.append(id)
            #title
            info.append(soup.find("span", class_= "post__title-text").text)
            #text
            text = soup.find("div", class_="post__text post__text-html post__text_v1").text
            info.append(cleanhtml(text))
            #date
            b = soup.find("span", class_= "post__time")
            c = b.text.split()[1]
            d = b.text.split()[2]
            if c == 'января':
                a = d + "-" +'01'
                info.append(a)
            elif c == 'ферваля':
                a = d + "-" +'02'
                info.append(a)
            elif c == 'марта':
                a = d + "-" +'03'
                info.append(a)
            elif c == 'апреля':
                a = d + "-" +'04'
                info.append(a)
            elif c == 'мая':
                a = d + "-" +'05'
                info.append(a)
            elif c == 'июня':
                a = d + "-" +'06'
                info.append(a)
            elif c == 'июля':
                a = d + "-" +'07'
                info.append(a)
            elif c == 'августа':
                a = d + "-" +'08'
                info.append(a)
            elif c == 'сентября':
                a = d + "-" +'09'
                info.append(a)
            elif c == 'октября':
                a = d + "-" +'10'
                info.append(a)
            elif c == 'ноября':
                a = d + "-" +'11'
                info.append(a)
            else:
                a = d + "-" +'12'
                info.append(a)
            #hubs
            tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
            hubs = ''
            for i in range(len(tagsshit)):
                hubs += tagsshit[i].text + ' '
            info.append(hubs)
            #tags
            tags = ''
            for tag in soup.find_all('li', class_= "inline-list__item inline-list__item_tag"):
                tags += tag.text + ' '
            info.append(tags)
            #votes
            a = soup.find(isvotes).text
            if a[0] == '–':
                b = int(a[1:])
                info.append(-b)
            else:
                info.append(int(a))
            #views
            a = soup.find("div", class_="post-stats__views").text.strip()
            if 'k' in a:
                views = float(a[:-1].replace(',', '.')) * 1000
            else:
                views = int(a)
            info.append(views)
            #bookmarks
            info.append(int(soup.find("span", class_="bookmark__counter js-favs_count").text))
            #comments
            cauthor = soup.find_all('span', class_="user-info__nickname user-info__nickname_small user-info__nickname_comment")
            author = soup.find('span', class_="user-info__nickname user-info__nickname_small").text
            allcauthors = [i.text.split() for i in cauthor]
            comment1 = soup.find_all("div", class_="comment__message")
            comments = [i.text for i in comment1]
            commentss = []
            for i in range(len(comments)):
                if comments[i] != 'НЛО прилетело и опубликовало эту надпись здесь':
                    commentss.append(comments[i])
            commentsitog = ''
            for i in range(len(commentss)):
                if allcauthors[i][0] != author:
                    commentsitog += commentss[i] + '~#'
            info.append(commentsitog)
        else:
            info.append('NaN')
    except:
            if page.status_code != 200:
                document(id)
    print(id)
    return info

### Функция для скачивания:

In [22]:
i = 'your number one'
for j in range('your number one + 100', 'your nubmer two', 100):
    b = map(document, np.arange(i, j))
    df = pd.DataFrame(b, columns=['id', 'title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
    df.to_feather('habr' + str(i) + "-" + str(j) + '.feather') 
    print('done', i, j)
    i += 100

### Функция для объединения:

In [ ]:
allhabr = pd.DataFrame(columns=['id', 'title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
i = #your number
for j in range('your number', 'your number', 100):
    new = pd.read_feather(str(i) + "-" + str(j) + '.feather')
    allhabr = allhabr.append(new, ignore_index=True)
    print('OK')
    i += 100

### Тест:

In [36]:
df = pd.DataFrame([document(493852)],columns=['id', 'title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])

493852


### Номера статей:

In [50]:
cnt = 207900
for i in ['Саша', 'Misha', 'Dasha', 'Anton', 'Dima']:
    print(i +',', 'your number 1:', str(cnt),'your number 2:', (cnt + a + 81))
    cnt += a + 80

Саша, your number 1: 207900 your number 2: 262801
Misha, your number 1: 262800 your number 2: 317701
Dasha, your number 1: 317700 your number 2: 372601
Anton, your number 1: 372600 your number 2: 427501
Dima, your number 1: 427500 your number 2: 482401
